In [ ]:
import numpy as np
from scipy.stats import dirichlet
np.set_printoptions(precision=2)

A large portion of the materials/code in this notebook are adapted from: [TIM HOPPER](https://dp.tdhopper.com/)

# Dirichlet Distribution

The Dirichlet distribution can be considered as a distribution of distributions. Each sample from the dirichlet distribution is a categorical distribution over $K$ categories. The distribution is parameterized using a $K$ dimensional vector $\alpha$. This parameter is also known as the concentration parameter. We can express the probability density function of the Dirichlet distribution as follows:

$$f(x_1, \dots, x_K; \alpha_1, \dots, \alpha_K) = \frac{1}{B(\alpha)} \prod_{i=1}^{K}x_{i}^{\alpha_{i} - 1}$$

where $\{x_{i}\}_{i=1}^{i=K}$ belong to the $K-1$ [simplex](https://en.wikipedia.org/wiki/Simplex). So $\sum_{i=1}^{K}x_{i} = 1$ and $x_{i} \geq 0 \ \forall i \in [1,K]$.

Here $B(\alpha)$ refers to the multivariate [Beta function](https://en.wikipedia.org/wiki/Beta_function) which we can express in terms of the [Gamma function](https://en.wikipedia.org/wiki/Gamma_function) as:
$$B(\alpha) = \frac{\prod_{i=1}^{K}\Gamma(\alpha_i)}{\Gamma(\sum_{i=1}^{K}a_i)}$$

The expected value of the Dirichlet distribution depends on the $\alpha$ vector and is given by:

$$\mathbb{E}[(x_{i}, \dots, x_{K})] = \frac{(a_{1}, \dots, a_{K})}{\sum_{i=1}^{K}a_{i}}$$

As the value (i.e. magnitude) of the concentration parameter $\alpha$ increases the standard deviation of the distribution decreases.

In [ ]:
def stats(scale_factor, G0=[1, 1, 1], N=10000):
    samples = dirichlet(alpha = scale_factor * np.array(G0)).rvs(N)
    print("                          alpha:", scale_factor)
    print("              element-wise mean:", samples.mean(axis=0))
    print("element-wise standard deviation:", samples.std(axis=0))
    print()

In the example below we draw 10000 samples from a symmetric Dirichlet distribution (i.e. one where each component of the $\alpha$ vector is equal).

Notice that as we increase the concentration parameter $\alpha$ the standard deviation of our distribution decreases. The mean value of the distribution remains thesame however.

In [ ]:
for alpha_val in [0.1, 1, 10, 100, 1000]:
    stats(alpha_val)

# Dirichlet Process

The Dirichlet Process can be seen as an infinite-dimensional generalization of the Dirichlet distribution. More concretely, in the same way as the Dirichlet distribution is the conjugate prior for the categorical distribution, the Dirichlet process is the conjugate prior for the infinite, nonparametric discrete distributions. For example, Dirichlet processes is a prior probability distribution for infinite mixture models.

A Dirichlet 